# DATA ANONYMIZATION

## Introduction

This notebook show how to use the anonymization feature with an small example. We will start by setting up the notebook, then we will create our dummy dataset and its metadata, and finally we will `model` and `sample` the data, checking the diferencies in both the data and the internal state of the objects.

## Notebook preparation

In [1]:
import json

import numpy as np
import pandas as pd
import rdt

from sdv.data_navigator import CSVDataLoader, Table
from sdv.modeler import Modeler
from sdv.sampler import Sampler

In [2]:
assert rdt.__version__ >= '0.1.2' , 'RDT minimal version that supports the feature'

## Creating dataset and metadata

We are going to create a dataset of a single table containing three different columns : `primary_key`, `name` and `credit_card_number` and two different metadata, one that does use anonymization, and the other that it doesn't.

In [3]:
# Generating data for table.
table_data = pd.DataFrame([
    {
        'primary_key': 1,
        'name': 'Bill',
        'credit_card_number': '1111-2222-3333-4444'
    },
    {
        'primary_key': 2,
        'name': 'Jeff',
        'credit_card_number': '0000-0000-0000-0000'
    },
    {
        'primary_key': 2,
        'name': 'Warren',
        'credit_card_number': '2222-2222-2222-2222'
    },
    {
        'primary_key': 2,
        'name': 'Bill',
        'credit_card_number': '9999-9999-9999-9999'
    },
    {
        'primary_key': 2,
        'name': 'Jeff',
        'credit_card_number': '8888-8888-8888-8888'
    },

]*10000) # We repeat the same values to smoth the modelling

# Storing as CSV
table_data.to_csv('table.csv', index=False, header=True)

Now we are going to generate the metadata. There are, a part from the anonymization parameters, two major differences with the other example metadata:



In [4]:
normal_table_name = 'normal'
normal_table_metadata = {
    'fields': [
        {
            'name': 'name', 
            'type': 'categorical', 
        },
        {
            'name': 'credit_card_number', 
            'type': 'categorical', 
        },
        {
            'name': 'primary_key', 
            'subtype': 'integer', 
            'type': 'number', 
            'regex': '^[0-9]{10}$'
        },

    ],
    'headers': True,
    'name': normal_table_name,
    'path': 'table.csv',
    'primary_key': 'primary_key',
    'use': True
}
normal_table_metadata

{'fields': [{'name': 'name', 'type': 'categorical'},
  {'name': 'credit_card_number', 'type': 'categorical'},
  {'name': 'primary_key',
   'subtype': 'integer',
   'type': 'number',
   'regex': '^[0-9]{10}$'}],
 'headers': True,
 'name': 'normal',
 'path': 'table.csv',
 'primary_key': 'primary_key',
 'use': True}

In [5]:
anon_table_name = 'anon'
anon_table_metadata = {
    'fields': [
        {
            'name': 'name', 
            'type': 'categorical',
            'pii': True,
            'pii_category': 'first_name'
        },
        {
            'name': 'credit_card_number', 
            'type': 'categorical',
            'pii': True,
            'pii_category': 'credit_card_number'
        },
        {
            'name': 'primary_key', 
            'subtype': 'integer', 
            'type': 'number', 
            'regex': '^[0-9]{10}$'
        },

    ],
    'headers': True,
    'name': anon_table_name,
    'path': 'table.csv',
    'primary_key': 'primary_key',
    'use': True
}
anon_table_metadata

{'fields': [{'name': 'name',
   'type': 'categorical',
   'pii': True,
   'pii_category': 'first_name'},
  {'name': 'credit_card_number',
   'type': 'categorical',
   'pii': True,
   'pii_category': 'credit_card_number'},
  {'name': 'primary_key',
   'subtype': 'integer',
   'type': 'number',
   'regex': '^[0-9]{10}$'}],
 'headers': True,
 'name': 'anon',
 'path': 'table.csv',
 'primary_key': 'primary_key',
 'use': True}

In [6]:
metadata = {
    'path': '',
    'tables': [
        anon_table_metadata,
        normal_table_metadata
    ]
}

metadata_filename = 'metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f)


Now we have all that we needed in order to model and sample our example dataset, that is:

- A table of data stored as `table.csv` file
- Two table metadata, both **using to the same table** , but only **one of them anonymizing data**, and each of them using a different name.
- A full metadata specification, including the table metadata mentioned above, stored as `metadata.json`

Now that we have prepared our data and metadata files is time to model and sample them. To do so, we will:

1. We will use `CSVDataLoader` to generate a `DataNavigator` instance for our dataset.
2. We will transform the data from our dataset in a format that can be modeled by `Modeler` by calling `data_navigator.transform_table()`.
3. After that, we will create a `Modeler` instance called `modeler`that will recieve as argument the `data_navigator`.
4. Then, we can finally **model** the dataset with `modeler.model_database`.
5. If the modelling is succesful, we can finally create a `Sampler` instance ready to sample new rows

In [10]:
# Loading data
data_loader = CSVDataLoader(metadata_filename)
data_navigator = data_loader.load_data()

# Let's check that everything has gone as expected.
assert anon_table_name in data_navigator.tables
assert normal_table_name in data_navigator.tables

# Let's check that the data for both tables is indeed the same.
assert data_navigator.tables[anon_table_name].data.equals(data_navigator.tables[normal_table_name].data)

# Transform data
data_navigator.transform_data()
modeler = Modeler(data_navigator)

# Model the dataset/database
modeler.model_database()
sampler = Sampler(data_navigator, modeler)

Now we are ready to samnple some data.

We will sample data from tables `anon` and `normal` that are originated from the same exact dataframe as we have confirmed before. The behavior that we are expecting is that on the anonymized table, unique values on the columns `credit_card_number` and `name` are not a subset of the unique values of the same columns in the original data table

In [12]:
anon_sampled = sampler.sample_rows('anon', 10)
anon_sampled

,credit_card_number,name,primary_key
0,3593118466800338,Susan,20
1,4484032365786036963,Marcus,21
2,3520900871832917,Susan,22
3,213128128484815,Susan,23
4,3593118466800338,Susan,24
5,3593118466800338,Susan,25
6,3520900871832917,Cathy,26
7,568403862630,Cathy,27
8,4484032365786036963,Marcus,28
9,568403862630,Cathy,29


Here we can see, that the `name` and `credit_card_number` have different values that on the original data, for exemple, the names have changed from ``


In [12]:
normal_sampled = sampler.sample_rows('normal', 10)
normal_sampled

,credit_card_number,name,primary_key
0,1111-2222-3333-4444,Bill,0
1,0000-0000-0000-0000,Bill,1
2,8888-8888-8888-8888,Bill,2
3,1111-2222-3333-4444,Jeff,3
4,9999-9999-9999-9999,Jeff,4
5,9999-9999-9999-9999,Jeff,5
6,1111-2222-3333-4444,Jeff,6
7,8888-8888-8888-8888,Bill,7
8,9999-9999-9999-9999,Bill,8
9,0000-0000-0000-0000,Jeff,9
